**Description**     
In this notebook, I'm exploring data downloaded via the Google analytics API.       
The uploaded data is converted to a DataFrame, which is then modified (data types, columns).        
The websites of interest are filtered out from the df.      
These are further evaluated (individually and in groups).       

**Required libraries**

In [103]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

**Data upload**

In [104]:
# Dataset containing data in the period - 2023-11-01 - 2024-11-30 
df = pd.read_json("../3_data/processed_data/df_analytics.json")

In [ ]:
df.head(5)

In [ ]:
df.info()

**Defining key KPIs**   
active_users    
new_users   
sessions_per_user   
average_session_duration    
engagement_rate = engaged_sessions / sessions   
new_users_ratio = new_users / active_users  

In [107]:
# Calculation of KPIs
df["engagement_rate"] = df["engaged_sessions"] / df["sessions"]
df["new_users_ratio"] = df["new_users"] / df["active_users"]


**Data processing**

In [108]:
# Float64 values rounding
columns_to_round = [
    "sessions_per_user",
    "average_session_duration",
    "engagement_rate",
    "new_users_ratio",
]

df[columns_to_round] = df[columns_to_round].round(2)

# Create a new column year_month
df.insert(loc=1, column='year_month', value=pd.to_datetime(df[['year', 'month']].assign(day=1)).dt.strftime('%Y-%m'))

**Data filtering**

In [109]:
# Websites of interest
websites = [
    "zemedelec.cz",
    "profipress.cz",
    "mechanizacezemedelstvi.cz", 
    "komunalweb.cz",
    "moderniobec.cz",
    "naschov.cz", 
    "odpady-online.cz",
    "vetweb.cz",
    "tvzemedelec.cz",
    "zahradaweb.cz",
]

# Data filtering
df_websites = df[df["website"].isin(websites)]

In [ ]:
df_websites.info()

**Data exploration**        
Variables and functions

In [111]:
metrics = [
    "active_users",
    "new_users",
    "sessions_per_user",
    "average_session_duration",
    "engagement_rate",
    "new_users_ratio",
]

In [112]:
sns.set_theme(
    style="whitegrid",
    palette="deep",
    font_scale=1.1,
    rc={"grid.color": "#e6e6e6"},
)

In [113]:
def get_report(df_website):
    """Functions for calculating and displaying average and total values.
    Display the months with the most number of active users"""
    
    print("Average per month:")
    for metric in metrics:
        print(f"  {metric}: {df_website[metric].mean().round(2)}")

    print("\nTotal per year:")
    for metric in ["active_users", "new_users"]:
        print(f"  {metric}: {df_website[metric].sum()}")

    top_rows = df_website.nlargest(3, 'active_users')
    return top_rows.head(3)

In [114]:
def get_report_charts(df_website):
    """Functions for creating graphs - FacetGrid for individual metrics, line chart for groups of metrics"""

    # 1) FacetGrid - line chart for each metric separately 
    df_melted = df_website.melt(id_vars=['year_month'], 
                                value_vars=metrics,
                                var_name='Metric', 
                                value_name='Value')

    grid = sns.FacetGrid(df_melted, col='Metric', col_wrap=2, sharey=False, height=4, aspect=1.5)
    grid.map(sns.lineplot, 'year_month', 'Value', linewidth=2)
    grid.set_titles("{col_name}") 
    grid.set_axis_labels("Date", "Value")
    grid.set(ylim=(0, None))

    for ax in grid.axes.flat:
        ax.tick_params(axis='x', rotation=45)  

    plt.suptitle("Trend of individual metrics over the year")
    plt.tight_layout()
    plt.show()

    # 2) Line chart - Active and new users
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=df_website, x="year_month", y="active_users", label="Active Users", linewidth=2)
    sns.lineplot(data=df_website, x="year_month", y="new_users", label="New Users")
    plt.title("Active and new users over the year")
    plt.xlabel("") 
    plt.ylabel("Number of users")
    plt.xticks(rotation=45, ha="right")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # 3) Line chart - Average session duration
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=df_website, x="year_month", y="average_session_duration", label="Average session duration", linewidth=2)
    plt.title("Average session duration over the year")
    plt.ylabel("Seconds")
    plt.xticks(rotation=45, ha="right")
    plt.xlabel("")
    plt.tight_layout()
    plt.show()

    # Line chart - User engagement metrics
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=df_website, x="year_month", y="sessions_per_user", label="Sessions per user", linewidth=2)
    sns.lineplot(data=df_website, x="year_month", y="engagement_rate", label="Engagement rate", linewidth=2)
    sns.lineplot(data=df_website, x="year_month", y="new_users_ratio", label="New Users ratio", linewidth=2)
    plt.title("User engagement metrics over year")
    plt.xticks(rotation=45, ha="right")  
    plt.xlabel("")  
    plt.ylabel("Ratio")
    plt.legend()
    plt.tight_layout()
    plt.show()

**Website zemedelec.cz**

In [115]:
df_zemedelec = df_websites[df_websites["website"] == "zemedelec.cz"]

In [ ]:
get_report(df_zemedelec)

In [ ]:
get_report_charts(df_zemedelec)

**Website profipress.cz**

In [118]:
df_profipress = df_websites[df_websites["website"] == "profipress.cz"]

In [ ]:
get_report(df_profipress)

In [ ]:
get_report_charts(df_profipress)

**Website mechanizacezemedelstvi.cz**

In [121]:
df_mechanizace = df_websites[df_websites["website"] == "mechanizacezemedelstvi.cz"]

In [ ]:
get_report(df_mechanizace)

In [ ]:
get_report_charts(df_mechanizace)

**Website komunalweb.cz**

In [124]:
df_komunalweb = df_websites[df_websites["website"] == "komunalweb.cz"]

In [ ]:
get_report(df_komunalweb)

In [ ]:
get_report_charts(df_komunalweb)

**Website moderniobec.cz**

In [127]:
df_moderniobec = df_websites[df_websites["website"] == "moderniobec.cz"]

In [ ]:
get_report(df_moderniobec)

In [ ]:
get_report_charts(df_moderniobec)

**Website naschov.cz**

In [130]:
df_naschov = df_websites[df_websites["website"] == "naschov.cz"]

In [ ]:
get_report(df_naschov)

In [ ]:
get_report_charts(df_naschov)

**Website odpady-online.cz**

In [133]:
df_odpady_online = df_websites[df_websites["website"] == "odpady-online.cz"]

In [ ]:
get_report(df_odpady_online)

In [ ]:
get_report_charts(df_odpady_online)

**Website vetweb.cz**

In [136]:
df_vetweb = df_websites[df_websites["website"] == "vetweb.cz"]

In [ ]:
get_report(df_vetweb)

In [ ]:
get_report_charts(df_vetweb)

**Website tvzemedelec.cz**

In [139]:
df_tvzemedelec = df_websites[df_websites["website"] == "tvzemedelec.cz"]

In [ ]:
get_report(df_tvzemedelec)

In [ ]:
get_report_charts(df_tvzemedelec)

**Website zahradaweb.cz**

In [142]:
df_zahradaweb = df_websites[df_websites["website"] == "zahradaweb.cz"]

In [ ]:
get_report(df_zahradaweb)

In [ ]:
get_report_charts(df_zahradaweb)

**Comparison of websites traffic**

In [145]:
sns.set_theme(
    style="whitegrid",
    palette="bright",
    font_scale=1.1,
    rc={"grid.color": "#e6e6e6"},
)

In [146]:
def comparison_sum_mean(df, metric):
    """Functions for creating charts comparing websites. Separate comparison for each metric.
    This function is used for metrics where the average value and sum of values is calculated."""

    metric_label = re.sub(r"_", " ", metric).title()

    # Line charts for websites
    plt.figure(figsize=(12,6))
    sns.lineplot(data=df, x="year_month", y=metric, hue="website", linewidth=3)
    plt.title(f"{metric_label} over time")
    plt.xticks(rotation=45, ha="right")
    plt.xlabel("")
    plt.ylabel(metric_label)
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0.)
    plt.tight_layout()
    plt.show()

    # Heatmap for websites
    df_pivot = df.pivot(index="website", columns="year_month", values=metric)
    plt.figure(figsize=(12, 6))
    sns.heatmap(df_pivot, cmap="viridis", annot=False, cbar_kws={"label": metric_label})
    plt.title(f"{metric_label} over time")
    plt.xlabel("")
    plt.ylabel("")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

    # Calculation of average and sum
    df_grouped_mean = df.groupby("website")[metric].mean().sort_values(ascending=False).reset_index()
    df_grouped_sum = df.groupby("website")[metric].sum().sort_values(ascending=False).reset_index()

    # Create a bar chart for average and sum
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    sns.barplot(data=df_grouped_mean, x=metric, y="website", color="lightgreen", ax=axes[0])
    axes[0].set_title(f"Average per month: {metric_label}")
    axes[0].set_xlabel(metric_label)
    axes[0].set_ylabel("")
    axes[0].tick_params(axis="x", rotation=45)

    sns.barplot(data=df_grouped_sum, x=metric, y="website", color="skyblue", ax=axes[1])
    axes[1].set_title(f"Total amount: {metric_label}")
    axes[1].set_xlabel(metric_label)
    axes[1].set_ylabel("")
    axes[1].tick_params(axis="x", rotation=45)

    plt.tight_layout()
    plt.show()

In [147]:
def comparison_mean(df, metric):
    """Functions for creating charts comparing websites. Separate comparison for each metric.
    This function is used for metrics where only the average value is calculated."""

    metric_label = re.sub(r"_", " ", metric).title()

    # Line charts for websites
    plt.figure(figsize=(12,6))
    sns.lineplot(data=df, x="year_month", y=metric, hue="website", linewidth=3)
    plt.title(f"{metric_label} over time")
    plt.xticks(rotation=45, ha="right")
    plt.xlabel("")
    plt.ylabel(metric_label)
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0.)
    plt.tight_layout()
    plt.show()

    # Heatmap for websites
    df_pivot = df.pivot(index="website", columns="year_month", values=metric)
    plt.figure(figsize=(12, 6))
    sns.heatmap(df_pivot, cmap="viridis", annot=False, cbar_kws={"label": metric_label})
    plt.title(f"{metric_label} over time")
    plt.xlabel("")
    plt.ylabel("")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

    # Calculation of average
    df_grouped_mean = df.groupby("website")[metric].mean().sort_values(ascending=False).reset_index()

    # Bar chart for average value
    plt.figure(figsize=(12, 6))
    sns.barplot(data=df_grouped_mean, x=metric, y="website", color="lightgreen")
    plt.title(f"Average per month: {metric_label}")
    plt.xlabel(metric_label)
    plt.ylabel("")
    plt.tight_layout()
    plt.show()

**Active users**

In [ ]:
comparison_sum_mean(df_websites, metric="active_users")

**New users**

In [ ]:
comparison_sum_mean(df_websites, "new_users")

**Sessions per user**

In [ ]:
comparison_mean(df_websites, "sessions_per_user")

**Average session duration**

In [ ]:
comparison_mean(df_websites, "average_session_duration")

**Engagement rate**

In [ ]:
comparison_mean(df_websites, "engagement_rate")

**New users ratio**

In [ ]:
comparison_mean(df_websites, "new_users_ratio")